# "Experimental" click2click co-visitation matrix

This notebook prepares one of click-to-click co-visitation matrixes. It is used to calculate features for the click model.
## Imports and definitions

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
  
import gc
from humanize import naturalsize

# functions and classes common for several notebooks of current project
import otto_common

In [2]:
class CalculateCovisitationMatrixExperiment(otto_common.CalculateCovisitationMatrix):
    '''
    Child class with logic specific to this type of covisitation matrix.
    It counts 20 aids after each aid if time between those events was less than 5 minutes.
    Weight value is calculated in a way that makes later events more important.
    '''

    def __init__(self, *args, timebase, **kwargs):
        self.timebase = timebase
        super().__init__(*args, **kwargs)

    def calculate_weights(self, df_i):
        df_i = df_i.sort_values(['session','ts'],ascending=[True,False])
        df_i = df_i.reset_index(drop=True)
        df_i['n'] = df_i.groupby('session').cumcount()
        for j in range(1,20):
            df_i2 = df_i.copy()
            df_i2['n'] = df_i2['n'] - j
            df_j = df_i.merge(df_i2,how='inner',on=['session', 'n'])
            if j > 2:
                df_j = df_j.loc[(df_j.ts_x - df_j.ts_y).abs() < 5 * 60]
            df_j = df_j[['session', 'aid_x', 'aid_y','ts_x']].drop_duplicates(['session', 'aid_x', 'aid_y']) 
            df_j['wgt'] = (1 + 3*(df_j.ts_x - self.timebase)/3024000) / j   
            df_j = df_j[['aid_x','aid_y','wgt']]
            if j == 1:
                df_j_all = df_j
            else:
                df_j_all = pd.concat([df_j_all, df_j], axis=0)
        del df_i, df_j, df_i2
        gc.collect()
        df_j_all = df_j_all.loc[df_j_all['aid_x'].isin(self.aid_list)]
        df_j_all.wgt = df_j_all.wgt.astype('float32')
        df_j_all = self.groupby_reset_and_reduce(df_j_all)
        return df_j_all

## Co-visitation matrix for cross-validation dataset

In [3]:
# Make a list of AIDs, that appear in cross-validation sets.
# aid_x is filtered to only include aids from the list, while aid_y is not filtered
# This reduces size of resulting matrix.
trunked_sessions_path = '/kaggle/input/otto-prepare-cv/cv_inputs.parquet'
trunked_sessions_path2 = '/kaggle/input/otto-prepare-cv/cv_inputs2.parquet'
cv_list = otto_common.build_aid_list(trunked_sessions_path, trunked_sessions_path2)

In [4]:
# Input parameters and matrix calculation itself.
click_data_cv = '/kaggle/input/otto-prepare-cv/cv_train.parquet'
n_splits = 150
n_splits_concat = 20
timebase_cv = 1658700000
covisitation_cv = CalculateCovisitationMatrixExperiment(n_splits, n_splits_concat, cv_list, timebase=timebase_cv)
df_matrix_cv = covisitation_cv.generate_covisitation_matrix(click_data_cv)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149


In [5]:
# Check dataframe size
size = df_matrix_cv.memory_usage(deep='True').sum()
print(naturalsize(size))
df_matrix_cv.to_parquet('matrix_experimental_cv.parquet')

del df_matrix_cv, covisitation_cv, cv_list
gc.collect()

3.2 GB


21

## Co-visitation matrix for test dataset
The same steps to create a co-visitation matrix for the test dataset.

In [6]:
trunked_sessions_path = '/kaggle/input/otto-prepare-cv/test.parquet'
test_list = otto_common.build_aid_list(trunked_sessions_path)

In [7]:
click_data_test = '/kaggle/input/otto-prepare-cv/train_full.parquet'
n_splits = 200
n_splits_concat = 25
timebase_test = 1659304800
covisitation_test = CalculateCovisitationMatrixExperiment(n_splits, n_splits_concat, test_list, timebase=timebase_test)
df_matrix_test = covisitation_test.generate_covisitation_matrix(click_data_test)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


In [8]:
# Check dataframe size
size = df_matrix_test.memory_usage(deep='True').sum()
print(naturalsize(size))

df_matrix_test.to_parquet('matrix_experimental_test.parquet')

3.9 GB
